In [145]:
import os, sys, glob, re, statistics, rich
from datetime import datetime
import pandas as pd

In [74]:
filesdir = "../results-validated/"
files = glob.glob(f"{filesdir}/*")

In [155]:
def getdate(datestr):
    ts = re.sub(r".*(2022.*) # .*", r"\1", datestr)
    return datetime.strptime(ts, '%Y-%m-%d %H:%M:%S,%f')

def mean_duration(hslines):
    diffs = []
    for s, e in hslines:
        init = getdate(s)
        end = getdate(e)
        diff = end - init
#         print(diff)
        diffs.append(diff.seconds + (diff.microseconds / 1000_000))
    if len(diffs) > 1:
        return round(statistics.mean(diffs), 3), round(statistics.stdev(diffs), 3)
    else:
        return round(statistics.mean(diffs), 3), -1

t = [('2022-06-05 15:49:07,819 # init handshake fe80::a4dd:f143:3055:dc42', '2022-06-05 15:49:12,068 # [init] end'),
    ('2022-06-05 15:49:20,500 # init handshake fe80::a4dd:f143:3055:dc42', '2022-06-05 15:49:22,500 # [init] end')]
mean_duration(t)

(3.124, 1.59)

In [156]:
NUM_MSG = 10
RPK_BYTES = 37 + 46 + 20
DID_BYTES = 37 + 63 + 37
titles = ["Auth.", "Total bytes", "Distance", "Roudtrip", "Standard dev.", "Success rate"]
res = []
for file in files[:]:
    if re.match(r".*2000.*", file):
        continue
#     print("\n\n", file)
    with open(file, "r") as f:
        log = f.readlines()
    hslines = [
        line.strip() for line in log
        if re.match(".*# init handshake.*", line) or re.match(".*\[init\] end", line)# or re.match(".*success.*", line)
    ]
#     print(hslines)
    hslines = list(zip(hslines[::2], hslines[1::2]))
    auth = re.sub(r".*edhoc-([A-Z]{3}).*", r"\1", file)
    value = [
        auth,
        DID_BYTES if auth == "DID" else RPK_BYTES,
        int(int(re.sub(r".*edhoc-[A-Z]{3}-(.*).log", r"\1", file)) / 100),
        mean_duration(hslines)[0],
        mean_duration(hslines)[1],
        len(hslines) / NUM_MSG,
    ]
    res.append(value)
#     print(value)
res = sorted(res, key=lambda e: e[1])
res = sorted(res, key=lambda e: e[0], reverse=True)
res
data = pd.DataFrame(res, columns=titles)
data

,Auth.,Total bytes,Distance,Roudtrip,Standard dev.,Success rate
0,RPK,103,1,5.020,0.002,1.0
1,RPK,103,5,5.022,0.004,1.0
2,RPK,103,15,5.024,0.012,1.0
3,RPK,103,10,5.027,0.013,1.0
4,DID,137,1,4.252,0.006,1.0
5,DID,137,5,4.252,0.005,1.0
6,DID,137,10,4.250,0.003,1.0
7,DID,137,15,4.452,0.633,1.0
